In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv('./Data/Fraud.csv')

In [4]:
# Starting With Data Analysis

In [5]:
df.columns

Index(['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud',
       'isFlaggedFraud'],
      dtype='object')

In [6]:
df.shape 
# this gives (rows, columns) present

(6362620, 11)

In [7]:
df.size
# Gives total no. of different data points

69988820

In [8]:
df.head()
# First 5 transactions data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [9]:
# We identify data types, we perform this step so to identigy object-type features. So to convert 
# them into numerical form for preprocessing step before training machine learning models
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [10]:
# formatting float-point numbers 'x' to diplay up to four decimal places
pd.set_option('display.float_format', lambda x: '%0.4f' %x)

In [11]:
round(df[['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig','oldbalanceDest',
                'newbalanceDest',]].describe() .T ,2)

,count,mean,std,min,25%,50%,75%,max
step,6362620.0000,243.4000,142.3300,1.0000,156.0000,239.0000,335.0000,743.0000
amount,6362620.0000,179861.9000,603858.2300,0.0000,13389.5700,74871.9400,208721.4800,92445516.6400
oldbalanceOrg,6362620.0000,833883.1000,2888242.6700,0.0000,0.0000,14208.0000,107315.1800,59585040.3700
newbalanceOrig,6362620.0000,855113.6700,2924048.5000,0.0000,0.0000,0.0000,144258.4100,49585040.3700
oldbalanceDest,6362620.0000,1100701.6700,3399180.1100,0.0000,0.0000,132705.6600,943036.7100,356015889.3500
newbalanceDest,6362620.0000,1224996.4000,3674128.9400,0.0000,0.0000,214661.4400,1111909.2500,356179278.9200


In [12]:
# Understanding relationship b/w input features and target features to select the important
# features for training machine learning model

df[['step', 'amount', 'oldbalanceOrg', 'newbalanceOrig','oldbalanceDest',
                'newbalanceDest','isFraud', 'isFlaggedFraud']].corr()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
step,1.0000,0.0224,-0.0101,-0.0103,0.0277,0.0259,0.0316,0.0033
amount,0.0224,1.0000,-0.0028,-0.0079,0.2941,0.4593,0.0767,0.0123
oldbalanceOrg,-0.0101,-0.0028,1.0000,0.9988,0.0662,0.0420,0.0102,0.0038
newbalanceOrig,-0.0103,-0.0079,0.9988,1.0000,0.0678,0.0418,-0.0081,0.0038
oldbalanceDest,0.0277,0.2941,0.0662,0.0678,1.0000,0.9766,-0.0059,-0.0005
newbalanceDest,0.0259,0.4593,0.0420,0.0418,0.9766,1.0000,0.0005,-0.0005
isFraud,0.0316,0.0767,0.0102,-0.0081,-0.0059,0.0005,1.0000,0.0441
isFlaggedFraud,0.0033,0.0123,0.0038,0.0038,-0.0005,-0.0005,0.0441,1.0000


In [13]:
# What we observed in above correlation matrix, is that ('oldbalanceDest', 'newbalanceDest') and 
# ('oldbalanceOrg', 'newbalanceOrg') indicates a high correlation (near to 1) or linear dependency between
# these pairs of features. And this could lead to issues in regression model.

# So, to fix this we need to remove one of the highly correated variables from each pair. 

In [14]:
df = df.drop(['newbalanceDest', 'newbalanceOrig'], axis=1)

In [15]:
df[df['amount']==0]

,step,type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,isFraud,isFlaggedFraud
2736447,212,CASH_OUT,0.0000,C1510987794,0.0000,C1696624817,0.0000,1,0
3247298,250,CASH_OUT,0.0000,C521393327,0.0000,C480398193,0.0000,1,0
3760289,279,CASH_OUT,0.0000,C539112012,0.0000,C1106468520,538547.6300,1,0
5563714,387,CASH_OUT,0.0000,C1294472700,0.0000,C1325541393,7970766.5700,1,0
5996408,425,CASH_OUT,0.0000,C832555372,0.0000,C1462759334,76759.9000,1,0
5996410,425,CASH_OUT,0.0000,C69493310,0.0000,C719711728,2921531.3400,1,0
6168500,554,CASH_OUT,0.0000,C10965156,0.0000,C1493336195,230289.6600,1,0
6205440,586,CASH_OUT,0.0000,C1303719003,0.0000,C900608348,1328472.8600,1,0
6266414,617,CASH_OUT,0.0000,C1971175979,0.0000,C1352345416,0.0000,1,0
6281483,646,CASH_OUT,0.0000,C2060908932,0.0000,C1587892888,0.0000,1,0


In [16]:
# Observation: Transactions which have 'amount'=0 are fraud transactions. We can also understand this thing as,
# if someone having 0 balance in their account, then their is no sense to transfer money. That is, probably some
# fraudster is targeting random accounts, and may be they don't aware about account balance.

In [17]:
df[df['isFlaggedFraud']==1]


,step,type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,isFraud,isFlaggedFraud
2736446,212,TRANSFER,4953893.0800,C728984460,4953893.0800,C639921569,0.0000,1,1
3247297,250,TRANSFER,1343002.0800,C1100582606,1343002.0800,C1147517658,0.0000,1,1
3760288,279,TRANSFER,536624.4100,C1035541766,536624.4100,C1100697970,0.0000,1,1
5563713,387,TRANSFER,4892193.0900,C908544136,4892193.0900,C891140444,0.0000,1,1
5996407,425,TRANSFER,10000000.0000,C689608084,19585040.3700,C1392803603,0.0000,1,1
5996409,425,TRANSFER,9585040.3700,C452586515,19585040.3700,C1109166882,0.0000,1,1
6168499,554,TRANSFER,3576297.1000,C193696150,3576297.1000,C484597480,0.0000,1,1
6205439,586,TRANSFER,353874.2200,C1684585475,353874.2200,C1770418982,0.0000,1,1
6266413,617,TRANSFER,2542664.2700,C786455622,2542664.2700,C661958277,0.0000,1,1
6281482,646,TRANSFER,10000000.0000,C19004745,10399045.0800,C1806199534,0.0000,1,1


In [18]:
# Observation: Transactions which are flag fraud by the system are actually fraud transactions

In [19]:
df[(df['amount']==df['oldbalanceOrg']) & (df['isFraud']==1)]

,step,type,amount,nameOrig,oldbalanceOrg,nameDest,oldbalanceDest,isFraud,isFlaggedFraud
2,1,TRANSFER,181.0000,C1305486145,181.0000,C553264065,0.0000,1,0
3,1,CASH_OUT,181.0000,C840083671,181.0000,C38997010,21182.0000,1,0
251,1,TRANSFER,2806.0000,C1420196421,2806.0000,C972765878,0.0000,1,0
252,1,CASH_OUT,2806.0000,C2101527076,2806.0000,C1007251739,26202.0000,1,0
680,1,TRANSFER,20128.0000,C137533655,20128.0000,C1848415041,0.0000,1,0
...,...,...,...,...,...,...,...,...,...
6362615,743,CASH_OUT,339682.1300,C786484425,339682.1300,C776919290,0.0000,1,0
6362616,743,TRANSFER,6311409.2800,C1529008245,6311409.2800,C1881841831,0.0000,1,0
6362617,743,CASH_OUT,6311409.2800,C1162922333,6311409.2800,C1365125890,68488.8400,1,0
6362618,743,TRANSFER,850002.5200,C1685995037,850002.5200,C2080388513,0.0000,1,0


In [20]:
# Observation: If transaction amount is same as amount present before transaction happended then the transaction 
# is basically a fraudulent transaction. Basically, we can say fraudsters know in this case that how much money is 
# present in the account of the victim and they directly clean the whole money from the account.